In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import keras

import tensorflow as tf

%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
keras.__version__, tf.__version__

In [ ]:
import os
from keras.backend.tensorflow_backend import set_session
os.environ["CUDA_VISIBLE_DEVICES"]="0"

config = tf.ConfigProto()
config.gpu_options.allow_growth=True
config.gpu_options.per_process_gpu_memory_fraction = 0.5
set_session(tf.Session(config=config))

### setup training

modify LABEL and images path dir. at pascal-keras-yolo.py
```
$ python pascal-keras-yolo3.py
$ mv train.txt keras-yolo3
```

In [ ]:
%run train.py

### edit yolo.py

since we will evaluate with mAP repo, we modify yolo.py to return predictions which will be used at predict_on_sample()

```
@@ -130,6 +130,7 @@ class YOLO(object):
                     size=np.floor(3e-2 * image.size[1] + 0.5).astype('int32'))
         thickness = (image.size[0] + image.size[1]) // 300
 
+        predictions = []
         for i, c in reversed(list(enumerate(out_classes))):
             predicted_class = self.class_names[c]
             box = out_boxes[i]
@@ -145,6 +146,15 @@ class YOLO(object):
             bottom = min(image.size[1], np.floor(bottom + 0.5).astype('int32'))
             right = min(image.size[0], np.floor(right + 0.5).astype('int32'))
             print(label, (left, top), (right, bottom))
+            
+            predictions.append({
+                'top': top,
+                'left': left,
+                'bottom': bottom,
+                'right': right,
+                'class': predicted_class,
+                'score': score
+            })
 
             if top - label_size[1] >= 0:
                 text_origin = np.array([left, top - label_size[1]])
@@ -164,7 +174,8 @@ class YOLO(object):
 
         end = timer()
         print(end - start)
-        return image
+        # return image
+        return predictions
```

In [ ]:
from yolo import YOLO
from yolo3.model import yolo_head, box_iou

from keras import backend as K
K.clear_session()

label = 'head' # head/shoulder/inria_person (body)
params = {
    'model_path': 'logs/000/' + label + '/trained_complete_final.h5', 
    'classes': 'model_data/hollywood_classes.txt', 
    'classes_path': 'model_data/inria_classes.txt',
    'image': True, 
    'output': ''
}
yolo = YOLO(**params)

In [ ]:
from PIL import Image

ImagesPath = '/home/lrs14/inria/INRIAPerson/Test/pos/'
TestFile = '/home/lrs14/inria/INRIAPerson/Test/pos.lst'
with open(TestFile) as f:
    sample = f.readlines()

label = 'head' # head/shoulder/inria_person

RESULTS_PATH = '/home/lrs14/KERAS-YOLO3/mAP/predicted/'
def predict_on_sample(yolo, sample):
    for val in sample:
        filename = val.split('/')[-1]
        filename = filename.replace('\n', '')
        filepath = ImagesPath + filename
        txt = open(RESULTS_PATH + filename.replace('png', 'txt'), 'w')
        image = Image.open(filepath)
        
        predictions = yolo.detect_image(image)
        for prediction in predictions:
            xmin = prediction['left']
            ymin = prediction['top']
            xmax = prediction['right']
            ymax = prediction['bottom']
            score = prediction['score']
            line = ' '.join(map(str, [label, score, xmin, ymin, xmax, ymax, '\n']))
            # uncomment when generating inference txt (1 class assumed)
            # print(' '.join(map(str, [xmin, ymin, xmax, ymax, score])))
            txt.write(line)
        txt.close()
        
predict_on_sample(yolo, sample)

#### parse the output to generate respective files at `inference-times` and `inferences`
